In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

C:\Users\lohani\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
train = pd.read_csv(r'C:\Users\lohani\Desktop\Books\Train.csv')
test = pd.read_csv(r'C:\Users\lohani\Desktop\Books\Test.csv')

In [8]:
# % MISSING VALUES

def missing(df):

    total = df.isnull().sum().sort_values(ascending = False)[df.isnull().sum().sort_values(ascending = False) != 0]
    percent = round(df.isnull().sum().sort_values(ascending = False)/len(df)*100,2)[round(df.isnull().sum().sort_values(ascending = False)/len(df)*100,2) != 0]
    return pd.concat([total, percent], axis=1, keys=['Total','Percent'])

display(missing(train))
display(missing(test))


,Total,Percent
X_2,138,1.72
X_1,138,1.72
product_quality_type,138,1.72
product_category_code,23,0.29
payment_emi_years(max),15,0.19
shipping_charges,14,0.17
price,10,0.12


,Total,Percent
product_quality_type,84,1.35
X_1,84,1.35
X_2,84,1.35
shipping_charges,24,0.39
product_category_code,16,0.26
price,15,0.24
payment_emi_years(max),14,0.23


## Train

In [10]:
# replacing null

train['X_2'] = train['X_2'].fillna(train['X_2'].mode()[0])
train['X_1'] = train['X_1'].fillna(train['X_1'].mode()[0])
train['product_quality_type'] = train['product_quality_type'].fillna(train['product_quality_type'].mode()[0])
train['product_category_code'] = train['product_category_code'].fillna(train['product_category_code'].mode()[0])
train['payment_emi_years(max)'] = train['payment_emi_years(max)'].fillna(train['payment_emi_years(max)'].mode()[0])
train['shipping_charges'] = train['shipping_charges'].fillna(train['shipping_charges'].mean())
train['price'] = train['price'].fillna(train['price'].mean())

In [13]:
# mapping labels

train['coupon_code'] = train['coupon_code'].map({'CLOTH15':1, 'CLOTH20':2, 'COSMETICS25':3, 'ELEC10':4
                                                 , 'ELEC15':5, 'FURN20':6, 'KIDS15':7})
train['payment_code'] = train['payment_code'].map({'APAY15':1, 'CITI10':2, 'GPAY20':3, 'HDFC20':4, 'PAYTM25':5})

In [14]:
from sklearn.preprocessing import LabelEncoder

train_product_quality_type = LabelEncoder()
train_order_status = LabelEncoder()

train['product_quality_type'] = train_product_quality_type.fit_transform(train['product_quality_type'])
train['order_status'] = train_order_status.fit_transform(train['order_status'])

In [15]:
train['order_checkout_date'] = pd.to_datetime(train['order_checkout_date'])

train['order_year'] = train['order_checkout_date'].dt.year
train['order_month'] = train['order_checkout_date'].dt.month
train['order_day'] = train['order_checkout_date'].dt.day
train['order_hour'] = train['order_checkout_date'].dt.hour
train['order_minute'] = train['order_checkout_date'].dt.minute
train['order_second'] = train['order_checkout_date'].dt.second
train['order_weekday'] = train['order_checkout_date'].dt.weekday
train['order_week'] = train['order_checkout_date'].dt.week
train.shape

(8009, 27)

In [16]:
train['delivery_eta'] = pd.to_datetime(train['delivery_eta'])

train['delivery_year'] = train['delivery_eta'].dt.year
train['delivery_month'] = train['delivery_eta'].dt.month
train['delivery_day'] = train['delivery_eta'].dt.day
train['delivery_weekday'] = train['delivery_eta'].dt.weekday
train['delivery_week'] = train['delivery_eta'].dt.week
train.shape

(8009, 32)

In [18]:
X = train.drop(['customer_id', 'seller_id', 'order_checkout_date', 'delivery_eta', 'coupon_code', 'payment_code'], axis=1)
y1 = train['coupon_code']
y2 = train['payment_code']

display(X.shape, y1.shape, y2.shape)

(8009, 26)

(8009,)

(8009,)

In [19]:
# splitting data

from sklearn.model_selection import train_test_split
X_train, X_validate, y1_train, y1_validate = train_test_split(X, y1, test_size=0.2, random_state=42)

In [20]:
# creating model

from sklearn.linear_model import LogisticRegression
model1 = LogisticRegression() 
model2 = LogisticRegression() 

In [21]:
model1.fit(X_train, y1_train)

C:\Users\lohani\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression()

In [22]:
y1_pred = model1.predict(X_validate)
y1_pred

array([5, 5, 5, ..., 4, 5, 5], dtype=int64)

In [23]:
# Evaluation
from sklearn.metrics import recall_score
recall_score(y1_validate, y1_pred, average='weighted')

0.5711610486891385

In [24]:
# splitting data

from sklearn.model_selection import train_test_split
X_train, X_validate, y2_train, y2_validate = train_test_split(X, y2, test_size=0.2, random_state=42)

In [25]:
model2.fit(X_train, y2_train)

C:\Users\lohani\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression()

In [26]:
y2_pred = model2.predict(X_validate)
y2_pred

array([2, 2, 2, ..., 2, 2, 2], dtype=int64)

In [27]:
# Evaluation
from sklearn.metrics import recall_score
recall_score(y2_validate, y2_pred, average='weighted')

0.7596754057428214

## Test

In [28]:
# replacing null

test['X_2'] = test['X_2'].fillna(test['X_2'].mode()[0])
test['X_1'] = test['X_1'].fillna(test['X_1'].mode()[0])
test['product_quality_type'] = test['product_quality_type'].fillna(test['product_quality_type'].mode()[0])
test['product_category_code'] = test['product_category_code'].fillna(test['product_category_code'].mode()[0])
test['payment_emi_years(max)'] = test['payment_emi_years(max)'].fillna(test['payment_emi_years(max)'].mode()[0])
test['shipping_charges'] = test['shipping_charges'].fillna(test['shipping_charges'].mean())
test['price'] = test['price'].fillna(test['price'].mean())

In [29]:
from sklearn.preprocessing import LabelEncoder

test_product_quality_type = LabelEncoder()
test_order_status = LabelEncoder()

test['product_quality_type'] = test_product_quality_type.fit_transform(test['product_quality_type'])
test['order_status'] = test_order_status.fit_transform(test['order_status'])

In [30]:
test['order_checkout_date'] = pd.to_datetime(test['order_checkout_date'])

test['order_year'] = test['order_checkout_date'].dt.year
test['order_month'] = test['order_checkout_date'].dt.month
test['order_day'] = test['order_checkout_date'].dt.day
test['order_hour'] = test['order_checkout_date'].dt.hour
test['order_minute'] = test['order_checkout_date'].dt.minute
test['order_second'] = test['order_checkout_date'].dt.second
test['order_weekday'] = test['order_checkout_date'].dt.weekday
test['order_week'] = test['order_checkout_date'].dt.week
test.shape

(6218, 25)

In [31]:
test['delivery_eta'] = pd.to_datetime(test['delivery_eta'])

test['delivery_year'] = test['delivery_eta'].dt.year
test['delivery_month'] = test['delivery_eta'].dt.month
test['delivery_day'] = test['delivery_eta'].dt.day
test['delivery_weekday'] = test['delivery_eta'].dt.weekday
test['delivery_week'] = test['delivery_eta'].dt.week
test.shape

(6218, 30)

In [33]:
X_test = test.drop(['customer_id', 'seller_id', 'order_checkout_date', 'delivery_eta'], axis=1)
X_test.shape

(6218, 26)

In [34]:
pred_cc = model1.predict(X_test)
pred_cc

array([5, 5, 5, ..., 5, 5, 5], dtype=int64)

In [35]:
pred_pc = model2.predict(X_test)
pred_pc

array([2, 2, 2, ..., 2, 2, 2], dtype=int64)

In [36]:
df1 = pd.DataFrame(test['customer_id'])
df2 = pd.DataFrame(pred_cc, columns=['coupon_code'])
df3 = pd.DataFrame(pred_pc, columns=['payment_code'])
display(df1.head(3), df2.head(3), df3.head(3))

,customer_id
0,d6b757c9478ed7366a42b6f09dce476f
1,9c224bc2354a4e8ecbd390cba06b5d07
2,5551ca919431fc78c6bb85ee6f57c904


,coupon_code
0,5
1,5
2,5


,payment_code
0,2
1,2
2,2


In [37]:
df = pd.concat([df1, df2, df3], axis=1)
df.head()

,customer_id,coupon_code,payment_code
0,d6b757c9478ed7366a42b6f09dce476f,5,2
1,9c224bc2354a4e8ecbd390cba06b5d07,5,2
2,5551ca919431fc78c6bb85ee6f57c904,5,2
3,77b4e2a15303756cc2a3b4a5d2dd3460,5,2
4,6b9c954ac26835161d0f34df89ca1175,4,2


In [38]:
# mapping labels

df['coupon_code'] = df['coupon_code'].map({1:'CLOTH15', 2:'CLOTH20', 3:'COSMETICS25', 4:'ELEC10'
                                                 , 5:'ELEC15', 6:'FURN20', 7:'KIDS15'})
df['payment_code'] = df['payment_code'].map({1:'APAY15', 2:'CITI10', 3:'GPAY20', 4:'HDFC20', 5:'PAYTM25'})

In [39]:
df.head()

,customer_id,coupon_code,payment_code
0,d6b757c9478ed7366a42b6f09dce476f,ELEC15,CITI10
1,9c224bc2354a4e8ecbd390cba06b5d07,ELEC15,CITI10
2,5551ca919431fc78c6bb85ee6f57c904,ELEC15,CITI10
3,77b4e2a15303756cc2a3b4a5d2dd3460,ELEC15,CITI10
4,6b9c954ac26835161d0f34df89ca1175,ELEC10,CITI10


In [40]:
df.to_csv(r'C:\Users\lohani\Desktop\dt11.csv', index=False)